### MAST30034: Applied Data Science Project 1
---
# Raw Data Analysis/Sandbox
#### Xavier Travers (1178369)

This is a cratchpad/sandbox notebook which has really simple checks.
NO DATA IS MODIFIED HERE.

In [1]:
# imports used throughout this notebook
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
import os
import re
from itertools import chain

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName('MAST30034 XT Project 1')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.sql.repl.eagerEval.enabled', True) 
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .getOrCreate()
)

22/08/07 17:16:38 WARN Utils: Your hostname, Polaris resolves to a loopback address: 127.0.1.1; using 172.28.235.139 instead (on interface eth0)
22/08/07 17:16:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/07 17:16:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/07 17:16:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/07 17:16:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/07 17:16:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


### Exploring the TLC Data

In [3]:
# read in one of the yellow pages
tlc_yellow_df = spark.read.parquet(f'../data/raw/tlc/yellow/2019-07.parquet')

In [4]:
tlc_yellow_df.schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', DoubleType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', IntegerType(), True)])

In [5]:
# add homemade helpers
sys.path.insert(1, '../scripts')
import helpers.cleaning_helpers as ch

ch.extract_day_month_year(tlc_yellow_df, 'tpep_pickup_datetime').limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,date,month,day,year
2,2019-07-01 10:51:04,2019-07-01 10:51:33,1.0,0.0,1.0,N,193,193,1,2.5,0.5,0.5,1.14,0.0,0.3,4.94,0.0,null,07/01/2019,7,1,2019
2,2019-07-01 10:46:04,2019-07-01 11:05:46,1.0,4.16,1.0,N,234,25,2,16.5,0.5,0.5,0.0,0.0,0.3,20.3,2.5,null,07/01/2019,7,1,2019
1,2019-07-01 10:25:09,2019-07-01 11:00:56,1.0,18.8,2.0,N,132,42,1,52.0,0.0,0.5,11.75,6.12,0.3,70.67,0.0,null,07/01/2019,7,1,2019
2,2019-07-01 10:33:32,2019-07-01 11:15:27,1.0,18.46,2.0,N,132,142,1,52.0,0.0,0.5,11.06,0.0,0.3,66.36,2.5,null,07/01/2019,7,1,2019
1,2019-07-01 10:00:55,2019-07-01 10:13:05,0.0,1.7,1.0,N,107,114,1,9.5,3.0,0.5,2.0,0.0,0.3,15.3,2.5,null,07/01/2019,7,1,2019
